In [18]:
import urllib.request
import pandas as pd
import requests
import io
import numpy as np
import seaborn as sns
import statsmodels.api as sm
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot


In [19]:
hsb2 = pd.read_csv("C:\\Users\\Admin\\Desktop\\Vaishnavi\\Auto\\NL.csv")

In [20]:
hsb2.head()

,Mode,Time,Cost,Hinc
0,1,15,7.5,9
1,0,30,2.5,9
2,0,20,3.5,9
3,0,10,7.5,7
4,1,15,2.5,7


In [21]:
hsb2.dtypes

Mode      int64
Time      int64
Cost    float64
Hinc      int64
dtype: object

In [22]:
hsb2["Cost"] = hsb2["Cost"].astype('int64')


In [23]:
hsb2.dtypes

Mode    int64
Time    int64
Cost    int64
Hinc    int64
dtype: object

In [24]:
hsb3 = hsb2


In [25]:
hsb3.head()

,Mode,Time,Cost,Hinc
0,1,15,7,9
1,0,30,2,9
2,0,20,3,9
3,0,10,7,7
4,1,15,2,7


In [29]:
y = hsb3['Mode']
X = hsb3.drop(['Mode'],axis=1)


In [28]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

# Define the softmax function
def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # Subtract max for numerical stability
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

# Define the negative log-likelihood function for multinomial logistic regression
def neg_log_likelihood(params, X, y, num_classes):
    num_features = X.shape[1]
    params = params.reshape((num_classes, num_features))
    logits = np.dot(X, params.T)
    probs = softmax(logits)
    log_likelihood = np.sum(y * np.log(probs))
    return -log_likelihood

# Define the prediction function
def predict(params, X, num_classes):
    num_features = X.shape[1]
    params = params.reshape((num_classes, num_features))
    logits = np.dot(X, params.T)
    probs = softmax(logits)
    return np.argmax(probs, axis=1)

# Assuming X and y are already defined
# Convert y to numpy array if it is a pandas Series
if isinstance(y, pd.Series):
    y = y.to_numpy()

# Add a small constant to X to avoid log(0) or log of negative values
small_constant = 1e-10
X_transformed = X + small_constant

# One-hot encode the target variable y
encoder = OneHotEncoder(sparse_output=False)
y_encoded = encoder.fit_transform(y.reshape(-1, 1))

# Get the number of classes and features
num_classes = y_encoded.shape[1]
num_features = X_transformed.shape[1]

# Initialize parameters
initial_params = np.zeros((num_classes, num_features))

# Minimize the negative log-likelihood
result = minimize(neg_log_likelihood, initial_params.ravel(), args=(X_transformed, y_encoded, num_classes), method='BFGS')
optimal_params = result.x

# Predict class labels using the optimized parameters
y_pred = predict(optimal_params, X_transformed, num_classes)

# Evaluate the model
accuracy = accuracy_score(y, y_pred)
print("Accuracy:", accuracy)

# Reshape the optimized parameters to get the intercepts and coefficients
optimal_params = optimal_params.reshape((num_classes, num_features))
print("Intercepts and Coefficients:", optimal_params)


Accuracy: 0.6666666666666666
Intercepts and Coefficients: [[ 0.01926213  0.01483844 -0.01439499]
 [-0.01975957 -0.01490095  0.01433571]]


In [30]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

# Define the softmax function
def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # Subtract max for numerical stability
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

# Define the negative log-likelihood function for multinomial logistic regression
def neg_log_likelihood(params, X, y, num_classes, num_features):
    intercepts = params[:num_classes]
    coefficients = params[num_classes:].reshape((num_classes, num_features))
    logits = np.dot(X, coefficients.T) + intercepts
    probs = softmax(logits)
    log_likelihood = np.sum(y * np.log(probs))
    return -log_likelihood

# Define the prediction function
def predict(params, X, num_classes, num_features):
    intercepts = params[:num_classes]
    coefficients = params[num_classes:].reshape((num_classes, num_features))
    logits = np.dot(X, coefficients.T) + intercepts
    probs = softmax(logits)
    return np.argmax(probs, axis=1)

# Assuming X and y are already defined
# Convert y to numpy array if it is a pandas Series
if isinstance(y, pd.Series):
    y = y.to_numpy()

# Add a small constant to X to avoid log(0) or log of negative values
small_constant = 1e-10
X_transformed = X + small_constant

# One-hot encode the target variable y
encoder = OneHotEncoder(sparse_output=False)
y_encoded = encoder.fit_transform(y.reshape(-1, 1))

# Get the number of classes and features
num_classes = y_encoded.shape[1]
num_features = X_transformed.shape[1]

# Initialize parameters (intercepts and coefficients)
initial_intercepts = np.zeros(num_classes)
initial_coefficients = np.zeros((num_classes, num_features))
initial_params = np.concatenate([initial_intercepts, initial_coefficients.ravel()])

# Minimize the negative log-likelihood
result = minimize(neg_log_likelihood, initial_params, args=(X_transformed, y_encoded, num_classes, num_features), method='BFGS')
optimal_params = result.x

# Predict class labels using the optimized parameters
y_pred = predict(optimal_params, X_transformed, num_classes, num_features)

# Evaluate the model
accuracy = accuracy_score(y, y_pred)
print("Accuracy:", accuracy)

# Extract intercepts and coefficients
optimal_intercepts = optimal_params[:num_classes]
optimal_coefficients = optimal_params[num_classes:].reshape((num_classes, num_features))

print("Intercepts:", optimal_intercepts)
print("Coefficients:", optimal_coefficients)


Accuracy: 0.6666666666666666
Intercepts: [-0.44745913  0.4474571 ]
Coefficients: [[ 0.02412407  0.02633354  0.02643919]
 [-0.02466459 -0.02640149 -0.02650095]]


In [14]:
from sklearn.base import BaseEstimator, ClassifierMixin, RegressorMixin

class Multinomial_Regression(BaseEstimator, ClassifierMixin): 
    def __init__(self, X, y,params=None):     
        if (params == None):
            self.learningRate = 0.005                  # Learning Rate
            self.max_epoch = 3000                      
        else:
            self.learningRate = params['LearningRate']
            self.max_epoch = params['Epoch'] # Epochs
        self.weight = np.array([[0.1,0.2,0.3],
                               [0.1,0.2,0.3],
                               [0.1,0.2,0.3],
                               [0.1,0.2,0.3]])

    def cost_derivate_gradient(self,n,Ti,Oi, X):
        result = -(np.dot(X.T,(Ti - Oi)))/n   
        return result 

    def function_cost_J(self,n,Ti,Oi):
        result = -(np.sum(Ti * np.log(Oi)))/n 
        return result
    def one_hot_encoding(self,Y):
        OneHotEncoding = []
        encoding = []
        for i in range(len(Y)):
            if(Y[i] == 0): encoding = np.array([1,0,0]) #Class 1, if y = 0
            elif(Y[i] == 1): encoding = np.array([0,1,0]) #Class 2, if y = 1
            elif(Y[i] == 2): encoding = np.array([0,0,1]) #Class 3, if y = 2

            OneHotEncoding.append(encoding)
        return OneHotEncoding
    
    def accuracy_graphic(self, answer_graph):
        labels = 'Hits', 'Faults'
        sizes = [96.5, 3.3]
        explode = (0, 0.14)
        fig1, ax1 = plt.subplots()
        ax1.pie(answer_graph, explode=explode, colors=['green','red'], labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
        ax1.axis('equal')
        plt.show()

    def softmax(self,z):
        soft = (np.exp(z).T / np.sum(np.exp(z),axis=1)).T 
        return soft
    
    def show_probability(self, arrayProbability):
        print("Probability: [ Class 0 ,  Class 1 , Class 2 ]")
        
        arrayTotal = []
        for k in arrayProbability:
            k[0] = "%.3f" % k[0]
            k[1] = "%.3f" % k[1]
            k[2] = "%.3f" % k[2]
            arrayTotal.append(k)
        for index, data in enumerate(arrayTotal):
            prob0 = data[0] * 100
            prob1 = data[1] * 100
            prob2 = data[2] * 100
            string = " {}: {}%, {}%, {}%".format(index, "%.3f" % prob0, "%.3f" % prob1, "%.3f" % prob2)
            print(string)
        
    def predict(self, X,y):
        acc_set = acc_vers = acc_virg = 0
        v_resp = []
        n = len(y)
        Z = np.matmul(X, self.weight)
        Oi = self.softmax(Z)
        prevision = np.argmax(Oi,axis=1)
        self.show_probability(Oi)
        print("")
        procent = sum(prevision == y)/n
        print(" ID-Sample  | Class Classification |  Output |   Hoped output  ")  
        for i in range(len(prevision)):
            if(prevision[i] == 0): print(" id :",i,"          | Iris-Setosa        |  Output:",prevision[i],"   |",y[i])
            elif(prevision[i] == 1): print(" id :",i,"          | Iris-Versicolour   |  Output:",prevision[i],"   |",y[i])
            elif(prevision[i] == 2): print(" id :",i,"          | Iris-Virginica     |  Output:",prevision[i],"   |",y[i])
                
        for i in range(len(prevision)):
            if((prevision[i] == y[i])and(prevision[i] == 0)):acc_set+=1
            elif((prevision[i] == y[i])and(prevision[i] == 1)):acc_vers+=1
            elif((prevision[i] == y[i])and(prevision[i] == 2)):acc_virg+=1
               
        correct = procent * 100
        incorrect = 100 - correct
        v_resp.append(correct)
        v_resp.append(incorrect)
        self.accuracy_graphic(v_resp)
        return "%.2f"%(correct), acc_set, acc_vers, acc_virg
    def show_err_graphic(self,v_epoch,v_error):
        plt.figure(figsize=(9,4))
        plt.plot(v_epoch, v_error, "m-")
        plt.xlabel("Number of Epoch")
        plt.ylabel("Error")
        plt.title("Error Minimization")
        plt.show()

    def fit(self,X,y):
        v_epochs = []
        totalError = []
        epochCount = 0
        n = len(X)
        gradientE = []
        while(epochCount < self.max_epoch):
            Ti = self.one_hot_encoding(y)
            Z = np.matmul(X,self.weight)
            Oi = self.softmax(Z)
            erro = self.function_cost_J(n,Ti,Oi)
            gradient = self.cost_derivate_gradient(n,Ti,Oi,X)
            self.weight = self.weight - self.learningRate * gradient
            if(epochCount % 100 == 0):
                totalError.append(erro)
                gradientE.append(gradient)
                v_epochs.append(epochCount)
                print("Epoch ",epochCount," Total Error:", "%.4f" % erro)
            
            epochCount += 1
        
        self.show_err_graphic(v_epochs,totalError)
        return self

In [17]:
arguments = {'Epoch':6000, 'LearningRate':0.005}
SoftmaxRegression = Multinomial_Regression(X,y,arguments)
SoftmaxRegression.fit(X,y)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\Admin\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_15116\3439214296.py", line 3, in <module>
    SoftmaxRegression.fit(X,y)
  File "C:\Users\Admin\AppData\Local\Temp\ipykernel_15116\412797928.py", line 107, in fit
    Z = np.matmul(X,self.weight)
        ^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Admin\anaconda3\Lib\site-packages\pandas\core\generic.py", line 2016, in __array_ufunc__
    return arraylike.array_ufunc(self, ufunc, method, *inputs, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Admin\anaconda3\Lib\site-packages\pandas\core\arraylike.py", line 273, in array_ufunc
    result = maybe_dispatch_ufunc_to_dunder_op(self, ufunc, method, *inputs, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [15]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import make_pipeline

# Add a small constant to X to avoid log(0) or log of negative values
small_constant = 1e-10
X_transformed = X + small_constant

# Apply log transformation to the data to follow a lognormal distribution
log_transformer = FunctionTransformer(np.log, validate=True)

# Create a pipeline that applies the log transformation followed by the multinomial logistic regression model
model = make_pipeline(
    log_transformer,
    LogisticRegression(multi_class='multinomial', solver='lbfgs')
)

# Fit the model to the data
model.fit(X_transformed, y)

# Predict the class labels for the same data
y_pred = model.predict(X_transformed)

# Evaluate the model
accuracy = accuracy_score(y, y_pred)
print("Accuracy:", accuracy)

# Extract the logistic regression model from the pipeline
logistic_model = model.named_steps['logisticregression']

print("Intercept:", logistic_model.intercept_)
print("Coefficients:", logistic_model.coef_)


Accuracy: 0.6666666666666666
Intercept: [0.40656766]
Coefficients: [[-0.26645862 -0.00132187]]


In [100]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.base import BaseEstimator, ClassifierMixin

class MultinomialLogisticRegression(BaseEstimator, ClassifierMixin):
    def __init__(self, max_iter=1000000):
        self.max_iter = max_iter
        self.coef_ = None
        self.intercept_ = None
        self.classes_ = None

    def softmax(self, Z):
        exp_Z = np.exp(Z - np.max(Z, axis=1, keepdims=True))
        return exp_Z / exp_Z.sum(axis=1, keepdims=True)

    def log_likelihood(self, params, X, y):
        n_samples, n_features = X.shape
        n_classes = len(np.unique(y))
        intercept = params[:n_classes]
        coef = params[n_classes:].reshape(n_classes, n_features)
        logits = X.dot(coef.T) + intercept
        probs = self.softmax(logits)
        log_likelihood = -np.sum(np.log(probs[np.arange(n_samples), y]))
        return log_likelihood

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)
        n_samples, n_features = X.shape
        self.classes_ = np.unique(y)
        n_classes = len(self.classes_)

        # Initial parameter values
        init_params = np.zeros(n_classes + n_classes * n_features)

        # Optimize the log-likelihood function
        result = minimize(self.log_likelihood, init_params, args=(X, y), method='L-BFGS-B', options={'maxiter': self.max_iter})
        
        # Extract the optimal parameters
        intercept = result.x[:n_classes]
        coef = result.x[n_classes:].reshape(n_classes, n_features)
        
        self.intercept_ = intercept
        self.coef_ = coef
        return self

    def predict_proba(self, X):
        logits = X.dot(self.coef_.T) + self.intercept_
        return self.softmax(logits)

    def predict(self, X):
        probs = self.predict_proba(X)
        return np.argmax(probs, axis=1)

# Example usage
# Assuming X and y are your features and target variables
# Encode the target variable if it's not already numeric
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Define the model
model = MultinomialLogisticRegression(max_iter=1000000)

# Define the model evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)

# Evaluate the model and collect the scores
n_scores = cross_val_score(model, X, y_encoded, scoring='accuracy', cv=cv, n_jobs=-1)

# Report the model performance
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

# Fit the model to the entire dataset
model.fit(X, y_encoded)
print("Intercepts:", model.intercept_)
print("Coefficients:", model.coef_)


Mean Accuracy: nan (nan)
Intercepts: [-0.24907489  0.24907505]
Coefficients: [[ 0.02642641  0.01748719]
 [-0.02643834 -0.01748854]]


In [101]:
summary = pd.DataFrame(zip(X.columns, np.transpose(model.coef_.tolist()[0])), 
                       columns=['features', 'coef'])
print(summary)

  features      coef
0     Time  0.026426
1     Cost  0.017487


In [13]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score



# Create a multinomial logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')

# Fit the model to the data
model.fit(X, y)

# Predict the class labels for the same data
y_pred = model.predict(X)

# Evaluate the model
accuracy = accuracy_score(y, y_pred)
print("Accuracy:", accuracy)
print(model.intercept_)
print(model.coef_)

Accuracy: 0.6666666666666666
[0.44547239]
[[-0.02438887 -0.02628912 -0.02624454]]


In [75]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

# Sample data


# Create a multinomial logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', 
                           C=1.0, max_iter=1000000)

# Fit the model to the data
model.fit(X, y)

# Get the log-likelihood of the fitted model
y_prob = model.predict_proba(X)
LL_model = -log_loss(y, y_prob, normalize=False)

# Fit the null model (only an intercept)
null_model = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', 
                                C=1.0, max_iter=1000000, fit_intercept=True)
X_null = np.ones((X.shape[0], 1))  # Only intercept
null_model.fit(X_null, y)

# Get the log-likelihood of the null model
y_null_prob = null_model.predict_proba(X_null)
LL_null = -log_loss(y, y_null_prob, normalize=False)

# Compute rho-squared
rho_squared = 1 - (LL_model / LL_null)
print("Rho-squared:", rho_squared)


Rho-squared: 0.8862496692559979


In [76]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

# Example: Adding polynomial features
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=42)

# Fit the model with polynomial features
model_poly = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', 
                                C=1.0, max_iter=1000000)
model_poly.fit(X_train, y_train)

# Evaluate the model
y_train_pred = model_poly.predict_proba(X_train)
y_test_pred = model_poly.predict_proba(X_test)
LL_model_train = -log_loss(y_train, y_train_pred, normalize=False)
LL_model_test = -log_loss(y_test, y_test_pred, normalize=False)

# Fit the null model
null_model_poly = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', 
                                     C=1.0, max_iter=1000000, fit_intercept=True)
null_model_poly.fit(np.ones((X_train.shape[0], 1)), y_train)

# Get the log-likelihood of the null model
y_null_train_pred = null_model_poly.predict_proba(np.ones((X_train.shape[0], 1)))
LL_null_train = -log_loss(y_train, y_null_train_pred, normalize=False)

# Compute rho-squared for training set
rho_squared_train = 1 - (LL_model_train / LL_null_train)
print("Rho-squared (train):", rho_squared_train)

# Evaluate on the test set
y_null_test_pred = null_model_poly.predict_proba(np.ones((X_test.shape[0], 1)))
LL_null_test = -log_loss(y_test, y_null_test_pred, normalize=False)

# Compute rho-squared for test set
rho_squared_test = 1 - (LL_model_test / LL_null_test)
print("Rho-squared (test):", rho_squared_test)


ValueError: y_true and y_pred contain different number of classes 2, 3. Please provide the true labels explicitly through the labels argument. Classes found in y_true: [1 3]

In [11]:
# define the multinomial logistic regression model with a default penalty
model = LogisticRegression(multi_class='multinomial', solver='lbfgs', penalty='l2', 
                           C=1.0, max_iter = 1000000)
# define the model evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate the model and collect the scores
n_scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
# report the model performance
print('Mean Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Mean Accuracy: 0.622 (0.166)


In [12]:
result = model.fit(X, y)

In [13]:
row = X.iloc[0:1, :]
# predict a multinomial probability distribution
yhat = model.predict_proba(row)
# summarize the predicted probabilities
print('Predicted Probabilities: %s' % yhat[0])

Predicted Probabilities: [0.63170614 0.36829386]


In [14]:
# predict the class label
yhat = model.predict(row)
# summarize the predicted class
print('Predicted Class: %d' % yhat[0])

Predicted Class: 0


In [15]:
print(result.intercept_)
print(result.coef_)

[0.2483642]
[[-0.02640554 -0.01743611]]


In [16]:
summary = pd.DataFrame(zip(X.columns, np.transpose(result.coef_.tolist()[0])), 
                       columns=['features', 'coef'])

In [17]:
print(summary)

  features      coef
0     Time -0.026406
1     Cost -0.017436


In [37]:
# Example dataset



# Fitting the MNLogit model
model = sm.MNLogit(y, X)
result = model.fit()

# Display the summary of the model
print(result.summary())


Optimization terminated successfully.
         Current function value: 0.600966
         Iterations 5
                          MNLogit Regression Results                          
Dep. Variable:                 Choice   No. Observations:                   30
Model:                        MNLogit   Df Residuals:                       28
Method:                           MLE   Df Model:                            1
Date:                Fri, 07 Jun 2024   Pseudo R-squ.:                 0.05585
Time:                        16:13:15   Log-Likelihood:                -18.029
converged:                       True   LL-Null:                       -19.095
Covariance Type:            nonrobust   LLR p-value:                    0.1442
  Choice=1       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Time          -0.0347      0.022     -1.546      0.122      -0.079       0.009
Cost          -0.0190      0.

In [38]:
row = X.iloc[0:1, :]
# predict a multinomial probability distribution
yhat = model.predict_proba(row)
# summarize the predicted probabilities
print('Predicted Probabilities: %s' % yhat[0])

AttributeError: 'MNLogit' object has no attribute 'predict_proba'

In [58]:
# Fitting the MNLogit model without an intercept
model = sm.MNLogit(y, X)
result = model.fit()

# Display the summary of the model
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.596139
         Iterations 6
                          MNLogit Regression Results                          
Dep. Variable:                 Choice   No. Observations:                   30
Model:                        MNLogit   Df Residuals:                       27
Method:                           MLE   Df Model:                            2
Date:                Thu, 06 Jun 2024   Pseudo R-squ.:                 0.06343
Time:                        22:20:42   Log-Likelihood:                -17.884
converged:                       True   LL-Null:                       -19.095
Covariance Type:            nonrobust   LLR p-value:                    0.2978
  Choice=1       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4982      0.954      0.522      0.601      -1.371       2.367
Time          -0.0529      0.

In [63]:
model = LogisticRegression(random_state=0, multi_class='multinomial', solver='newton-cg')
result = model.fit(X, y)
print(result.intercept_)
print(result.coef_)

[0.24836398]
[[-0.02640551 -0.01743616]]


In [67]:
# Fitting the MNLogit model without an intercept using Newton-Raphson method
X = sm.add_constant(X)
model = sm.MNLogit(y, X)
result = model.fit(method='newton')

# Display the summary of the model
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.596139
         Iterations 6
                          MNLogit Regression Results                          
Dep. Variable:                 Choice   No. Observations:                   30
Model:                        MNLogit   Df Residuals:                       27
Method:                           MLE   Df Model:                            2
Date:                Thu, 06 Jun 2024   Pseudo R-squ.:                 0.06343
Time:                        22:33:38   Log-Likelihood:                -17.884
converged:                       True   LL-Null:                       -19.095
Covariance Type:            nonrobust   LLR p-value:                    0.2978
  Choice=1       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4982      0.954      0.522      0.601      -1.371       2.367
Time          -0.0529      0.

In [68]:
!pip install xlogit

  Obtaining dependency information for xlogit from https://files.pythonhosted.org/packages/5b/ab/5280d6920d59e739063effb59b54349a71ee66bdd11daa1664d08e68564a/xlogit-0.2.7-py3-none-any.whl.metadata


In [19]:
p1= []
p2 = []

# Initialize an empty list to store predicted probabilities
for i in range(len(hsb2.Mode)):
    row = X.iloc[i:i+1, :]  # Select a single row from the DataFrame
    # Predict a multinomial probability distribution
    yhat = model.predict_proba(row)
    # Summarize the predicted probabilities and append to the list
    p1.append(yhat[0][0])
    p2.append(yhat[0][1])
  
hsb2['P1'] = p1
hsb2['P2'] = p2


In [21]:
# predict the class label
yhat = model.predict(row)
# summarize the predicted class
print('Predicted Class: %d' % yhat[0])

c = []
# Initialize an empty list to store predicted probabilities
for i in range(len(hsb2.Mode)):
    row = X.iloc[i:i+1, :]  # Select a single row from the DataFrame
    # Predict a multinomial probability distribution
    yhat = model.predict(row)
    # Summarize the predicted probabilities and append to the list
    c.append(yhat[0])
    

hsb2['Predicted'] = c

Predicted Class: 0


In [24]:
print(hsb2)

    Person No.  Mode  Time  Cost  Choice        P1        P2  Predicted
0            1     1    15     7       1  0.631706  0.368294          0
1            1     2    30     0       0  0.747930  0.252070          0
2            1     3    20     3       0  0.660183  0.339817          0
3            2     1    10     7       0  0.568440  0.431560          0
4            2     2    15     0       1  0.573327  0.426673          0
5            2     3    10     3       0  0.533947  0.466053          0
6            3     1     5     7       0  0.502858  0.497142          0
7            3     2    20     0       0  0.636335  0.363665          0
8            3     3     7     3       1  0.494392  0.505608          1
9            4     1    25    25       0  0.844927  0.155073          0
10           4     2    60     0       0  0.935350  0.064650          0
11           4     3    20     5       1  0.675650  0.324350          0
12           5     1    10     7       0  0.568440  0.431560    